In [1]:
%reload_ext autoreload
%autoreload
%matplotlib inline
import os, json,glob,sys,io
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats, eval_model
import pickle
from datetime import datetime 
import torch
from torch import nn
from transformers import BertTokenizer
from model.MedClinical import Biobert_fc 
from sklearn.metrics import classification_report, f1_score
from util.tools  import load_config
from util.DataLoader  import *
from pathlib import Path, PureWindowsPath, PurePosixPath
import traceback
from matplotlib import pyplot as plt
import pandas as pd

In [8]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [9]:
config_folder = "config/"
config = load_config(config_folder)

In [10]:
dataprocessor = MultiClassificationProcessor()
test_dataloader, dev_data_len, dev_num_labels, dev_num_train_optimization_steps, all_dev_label_ids = dataprocessor.get_data_loader(config,source='test')    

INFO:root:***** Running training *****
INFO:root:  Num examples = 1000
INFO:root:  Batch size = 24
INFO:root:  Num steps = 410


In [11]:
def save_missed_cases_to_file(file_name, preds, label_ids, inputs):
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
    missed_cases = []
    all_cases = []
    for i in range(0,len(label_ids)):
        all_cases.append([ file_name, preds[i],  label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(inputs[i])) ])
        if label_ids[i] !=  preds[i]:
            missed_cases.append([ file_name, preds[i],  label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(inputs[i])) ])
    #Save into a file
    missed_cases_file = config.programsettings.REPORTS_DIR +'missed_test_results_' + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    all_cases_file = config.programsettings.REPORTS_DIR +'all_test_results_' + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    
    with open(missed_cases_file, "wb") as f:
        pickle.dump(missed_cases, f)  
        
    with open(all_cases_file, "wb") as f:
        pickle.dump(all_cases, f)  
        
    

In [12]:
models_folder_name = config.programsettings.OUTPUT_DIR
data_folder = Path(models_folder_name)
list_model_files = list(data_folder.glob('*.bin'))
num_labels = 9

f1_score_list = []
for model_file in list_model_files:
    try:
        print("\n Model file: ", model_file)
        model = torch.load(model_file)
#         model = nn.DataParallel(model)
        x =model.eval()
        test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)  
        print(classification_report(test_labels,test_preds ))
        print("\n \n \n")
        f1_value = f1_score(test_labels, test_preds,average='macro')
        f1_score_list.append([model_file,f1_value ])
        file_name = model_file.__str__().replace(".", "_") 
        save_missed_cases_to_file( file_name, test_preds, test_labels, test_inputs)
        
    except:
#         print("Failed due to cpu & GPU incompatibility")
     # printing stack trace 
        traceback.print_exc()         

print("\n \n \n")





 Model file:  outputs\re\BioBERT_CNN_fc2020-11-23 20_32_17_598176.bin




 Model file:  outputs\re\BioBERT_CNN_fc2020-11-23 23_16_23_167740.bin


Traceback (most recent call last):
  File "<ipython-input-12-c3208afc4473>", line 13, in <module>
    test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)
  File "C:\prabhu\edu\code\w266\final_project\project_re\eval.py", line 38, in eval_model
    logits = model(input_ids, segment_ids, input_mask)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "C:\prabhu\edu\code\w266\final_project\project_re\model\MedClinical.py", line 67, in forward
    sequence_output, pooled_output = self.bert(
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\parallel\data_parallel.py", line 157, in forward
    inputs, kwargs = self.scatter(inputs, kwargs, self.device_ids)
  File "C:\ProgramD



 Model file:  outputs\re\BioBERT_CNN_fc2020-11-24 03_13_56_562308.bin


Traceback (most recent call last):
  File "<ipython-input-12-c3208afc4473>", line 13, in <module>
    test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)
  File "C:\prabhu\edu\code\w266\final_project\project_re\eval.py", line 38, in eval_model
    logits = model(input_ids, segment_ids, input_mask)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "C:\prabhu\edu\code\w266\final_project\project_re\model\MedClinical.py", line 67, in forward
    sequence_output, pooled_output = self.bert(
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\parallel\data_parallel.py", line 157, in forward
    inputs, kwargs = self.scatter(inputs, kwargs, self.device_ids)
  File "C:\ProgramD

Traceback (most recent call last):
  File "<ipython-input-12-c3208afc4473>", line 13, in <module>
    test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)
  File "C:\prabhu\edu\code\w266\final_project\project_re\eval.py", line 38, in eval_model
    logits = model(input_ids, segment_ids, input_mask)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "C:\prabhu\edu\code\w266\final_project\project_re\model\MedClinical.py", line 67, in forward
    sequence_output, pooled_output = self.bert(
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\parallel\data_parallel.py", line 157, in forward
    inputs, kwargs = self.scatter(inputs, kwargs, self.device_ids)
  File "C:\ProgramD

In [ ]:
pd.set_option('display.max_colwidth', 120)
columns = ['file_namae', 'f1-score']
test_results= pd.DataFrame(f1_score_list, columns = columns)
test_results

In [ ]:
# print ("F1 Score summary \n")  
# ax = plt.gca()
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)
# plt.box(on=None)
# plt.figure(figsize=(14,6))
# the_table = plt.table(cellText=f1_score_list,
#                       colLabels=['file name', 'F1-score'],
#                       loc='center')
# the_table.auto_set_font_size(False)
# the_table.set_fontsize(10)